(sec:chelpg_charges)=
# CHELPG charges

Different choices of grid points in the [Merz–Kollman (MK) scheme](sec:esp_charges) can be made. CHELPG charges are obtained with grid points chosen on a dense cubic grid with exclusion made of grid points inside the van der Waals molecular volume {cite}`Brennan1990`.

In contrast to the original MK scheme, the calculation of CHELPG charges involve grid points directly outside the van der Waals molecular volume, and since the electrostatic potential is here large, these points will be important for the minimization of the Lagrangian. We note that there is no universal grid-point choice that can be considered best for all situations. 

Let us determine the CHELPG charges of methanol at the Hartree–Fock/6-31G(d) level of theory.

In [1]:
methanol_xyz = """6

  H      1.2001      0.0363      0.8431
  C      0.7031      0.0083     -0.1305
  H      0.9877      0.8943     -0.7114
  H      1.0155     -0.8918     -0.6742
  O     -0.6582     -0.0067      0.1730
  H     -1.1326     -0.0311     -0.6482
"""

In [2]:
import py3Dmol as p3d

viewer = p3d.view(width=300, height=200)

viewer.addModel(methanol_xyz, "xyz")
viewer.setStyle({"stick": {}})

viewer.show()

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [3]:
import veloxchem as vlx

* Warning * Environment variable OMP_NUM_THREADS not set.
* Warning * Setting OMP_NUM_THREADS to 6.


First, we determine the reference state of the system.

In [4]:
molecule = vlx.Molecule.read_xyz_string(methanol_xyz)
basis = vlx.MolecularBasis.read(molecule, "6-31G*", ostream=None)

scf_drv = vlx.ScfRestrictedDriver()
scf_drv.ostream.mute()

scf_results = scf_drv.compute(molecule, basis)

Second, we calculate CHELPG charges with the `compute` method of the `RespChargesDriver` class.

In [5]:
esp_drv = vlx.RespChargesDriver()

esp_drv.grid_type = "chelpg"

esp_drv.equal_charges = "1=3, 1=4"

chelpg_charges = esp_drv.compute(molecule, basis, scf_results, "esp")

                                                                                                                          
                                                 ESP Charges Driver Setup                                                 
                                                                                                                          
                                         Number of Conformers         :  1                                                
                                         Grid Spacing in Angstrom     :  0.3                                              
                                         Grid Margin in Angstrom      :  2.8                                              
                                         Total Number of Grid Points  :  5890                                             
                                                                                                                          
                

Third, we print out the results.

In [6]:
print("Atom   CHELPG charge")

print(20 * "-")

for label, chelpg_charge in zip(molecule.get_labels(), chelpg_charges):
    print(f"{label :s} {chelpg_charge : 18.6f}")

print(20 * "-")

print(f"Total: {chelpg_charges.sum() : 13.6f}")

Atom   CHELPG charge
--------------------
H           0.003200
C           0.225480
H           0.003200
H           0.003200
O          -0.611345
H           0.376265
--------------------
Total:      0.000000


Compared to the ESP results, the positive charge of the methyl group is here found to be more localized to the carbon atom.